We need to compile llama-cpp-python[server] using Nvidia Compile (nvcc). Make sure you are running a CUDA Notebook !

In [10]:
!NVCC_APPEND_FLAGS='-allow-unsupported-compiler' CUDACXX=/usr/local/cuda/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python[server] --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 148.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 214.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 247.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 151.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 168.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 268.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 273.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 246.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 236.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 249.6 MB/s eta 0:

Next, install the hugging face CLI and Torch tool so we can grab a llama model to chat to.

In [ ]:
pip install 'huggingface_hub[cli,torch]'

Login to Hugging Face, you will need your API_KEY set as an environment variable. 

In [ ]:
from huggingface_hub import login, logout
import os
login(os.getenv("HF_API_KEY"))

Download a llama-3 model. This has a few quantized, finetuned
versions - https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF

In [ ]:
!huggingface-cli download lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF --local-dir . --local-dir-use-symlinks False

Load the model making sure to specify to use the GPU with n_gpu_layers.

In [ ]:
from llama_cpp import Llama
llm = Llama(model_path="Meta-Llama-3-8B-Instruct-Q8_0.gguf", n_gpu_layers=-1)

This is where we specify the completions.

In [ ]:
output = llm(
  "<PROMPT>", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

In [ ]:
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant with an Australian style."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)

In [ ]:
!python3 -m llama_cpp.server --model Meta-Llama-3-8B-Instruct-Q8_0.gguf --n_gpu_layers=-1 --host=0.0.0.0 --port=8080

Try browsing to the Route and Proxy that your notebook exposes. Be sure to create the two Routes in OpenShift first.

```yaml
oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-openapi-json
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/openapi.json
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /openapi.json
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF

oc apply -n rhods-notebooks -f - <<EOF
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: llama-v1
  annotations:
    haproxy.router.openshift.io/rewrite-target: /notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/v1
spec:
  host: jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com
  path: /v1
  port:
    targetPort: oauth-proxy
  tls:
    termination: Reencrypt
    insecureEdgeTerminationPolicy: None
  to:
    kind: Service
    name: jupyter-nb-admin-tls
    weight: 100
  wildcardPolicy: None
EOF
```


Open in a browser tab and try it out using swagger.

https://jupyter-nb-admin-rhods-notebooks.apps.sno.sandbox.opentlc.com/notebook/rhods-notebooks/jupyter-nb-admin/proxy/8080/docs